<a href="https://colab.research.google.com/github/qmohsu/unperturbed_satellite_orbit_simulation/blob/main/unperturbed_satellite_orbit_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

# Constants
G = 6.67430e-11  # Gravitational constant (m^3 kg^-1 s^-2)
M_earth = 5.972e24  # Earth mass (kg)
R_earth = 6371  # Earth radius (km)
mu = G * M_earth  # Earth's gravitational parameter (m^3 s^-2)

def keplerian_to_cartesian(a, e, i, RAAN, omega, M, mu=mu):
    """
    Convert Keplerian elements to Cartesian position and velocity vectors.

    Parameters:
    a: Semi-major axis (km)
    e: Eccentricity
    i: Inclination (degrees)
    RAAN: Right Ascension of the Ascending Node (degrees)
    omega: Argument of Perigee (degrees)
    M: Mean Anomaly (degrees)
    mu: Gravitational parameter (m^3 s^-2)

    Returns:
    r, v: Position and velocity vectors in Earth-centered inertial frame (km, km/s)
    """
    # Convert angles from degrees to radians
    i = np.radians(i)
    RAAN = np.radians(RAAN)
    omega = np.radians(omega)
    M = np.radians(M)

    # Calculate eccentric anomaly using Newton-Raphson method
    E = M
    for _ in range(10):  # Usually converges within a few iterations
        E = E - (E - e * np.sin(E) - M) / (1 - e * np.cos(E))

    # Calculate true anomaly
    nu = 2 * np.arctan2(np.sqrt(1 + e) * np.sin(E/2), np.sqrt(1 - e) * np.cos(E/2))

    # Calculate distance from central body
    r_mag = a * (1 - e * np.cos(E))

    # Calculate position and velocity in orbital plane
    r_orbital = np.array([
        r_mag * np.cos(nu),
        r_mag * np.sin(nu),
        0
    ])

    v_orbital = np.array([
        -np.sqrt(mu / a) * np.sin(E),
        np.sqrt(mu / a) * np.sqrt(1 - e**2) * np.cos(E),
        0
    ])

    # Rotation matrices
    R3_omega = np.array([
        [np.cos(omega), -np.sin(omega), 0],
        [np.sin(omega), np.cos(omega), 0],
        [0, 0, 1]
    ])

    R1_i = np.array([
        [1, 0, 0],
        [0, np.cos(i), -np.sin(i)],
        [0, np.sin(i), np.cos(i)]
    ])

    R3_RAAN = np.array([
        [np.cos(RAAN), -np.sin(RAAN), 0],
        [np.sin(RAAN), np.cos(RAAN), 0],
        [0, 0, 1]
    ])

    # Combined rotation matrix
    R = R3_RAAN @ R1_i @ R3_omega

    # Transform to Earth-centered inertial frame
    r = R @ r_orbital
    v = R @ v_orbital

    return r, v

def propagate_orbit(a, e, i, RAAN, omega, M, num_points=1000):
    """
    Propagate orbit and return points along the orbit.

    Parameters:
    a: Semi-major axis (km)
    e: Eccentricity
    i: Inclination (degrees)
    RAAN: Right Ascension of the Ascending Node (degrees)
    omega: Argument of Perigee (degrees)
    M: Mean Anomaly (degrees)
    num_points: Number of points to generate

    Returns:
    xs, ys, zs: Lists of coordinates for points along the orbit
    """
    # Generate points along the orbit
    xs, ys, zs = [], [], []

    for M_point in np.linspace(0, 360, num_points):
        r, _ = keplerian_to_cartesian(a, e, i, RAAN, omega, M_point)
        xs.append(r[0])
        ys.append(r[1])
        zs.append(r[2])

    return xs, ys, zs

def visualize_orbit(a, e, i, RAAN, omega, M, animate=False):
    """
    Visualize the orbit with the given Keplerian elements.

    Parameters:
    a: Semi-major axis (km)
    e: Eccentricity
    i: Inclination (degrees)
    RAAN: Right Ascension of the Ascending Node (degrees)
    omega: Argument of Perigee (degrees)
    M: Mean Anomaly (degrees)
    animate: Whether to animate the orbit
    """
    # Propagate the orbit
    xs, ys, zs = propagate_orbit(a, e, i, RAAN, omega, M)

    # Create a figure
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plot the Earth
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    x = R_earth * np.cos(u) * np.sin(v)
    y = R_earth * np.sin(u) * np.sin(v)
    z = R_earth * np.cos(v)
    ax.plot_surface(x, y, z, color='blue', alpha=0.3)

    # Plot the orbit
    orbit_line, = ax.plot(xs, ys, zs, color='red', label='Orbit')

    # Plot the current position
    r, _ = keplerian_to_cartesian(a, e, i, RAAN, omega, M)
    satellite, = ax.plot([r[0]], [r[1]], [r[2]], 'ko', markersize=6, label='GPS Satellite')

    # Plot the perigee
    r_perigee, _ = keplerian_to_cartesian(a, e, i, RAAN, omega, 0)
    ax.plot([r_perigee[0]], [r_perigee[1]], [r_perigee[2]], 'go', markersize=6, label='Perigee')

    # Plot the apogee
    r_apogee, _ = keplerian_to_cartesian(a, e, i, RAAN, omega, 180)
    ax.plot([r_apogee[0]], [r_apogee[1]], [r_apogee[2]], 'bo', markersize=6, label='Apogee')

    # Plot the coordinate axes
    max_val = max(max(abs(np.array(xs))), max(abs(np.array(ys))), max(abs(np.array(zs)))) * 1.2
    ax.plot([0, max_val], [0, 0], [0, 0], 'k-', alpha=0.5, linewidth=1)
    ax.plot([0, 0], [0, max_val], [0, 0], 'k-', alpha=0.5, linewidth=1)
    ax.plot([0, 0], [0, 0], [0, max_val], 'k-', alpha=0.5, linewidth=1)
    ax.text(max_val*1.1, 0, 0, 'X', fontsize=10)
    ax.text(0, max_val*1.1, 0, 'Y', fontsize=10)
    ax.text(0, 0, max_val*1.1, 'Z', fontsize=10)

    # Plot the ascending node
    ascending_node_lon = RAAN
    ax.plot([R_earth * np.cos(np.radians(ascending_node_lon))],
            [R_earth * np.sin(np.radians(ascending_node_lon))],
            [0], 'mo', markersize=6, label='Ascending Node')

    # Plot the orbital plane
    if e < 1:  # Only for elliptical orbits
        # Create a meshgrid in the orbital plane
        u = np.linspace(0, 2 * np.pi, 100)
        v = np.linspace(0, 1, 50) * a * (1 - e**2)

        # Create points in a circular disk in the orbital plane
        plane_x = np.outer(np.cos(u), v)
        plane_y = np.outer(np.sin(u), v)
        plane_z = np.zeros_like(plane_x)

        # Rotate the disk to match the orbital plane
        R3_omega = np.array([
            [np.cos(np.radians(omega)), -np.sin(np.radians(omega)), 0],
            [np.sin(np.radians(omega)), np.cos(np.radians(omega)), 0],
            [0, 0, 1]
        ])

        R1_i = np.array([
            [1, 0, 0],
            [0, np.cos(np.radians(i)), -np.sin(np.radians(i))],
            [0, np.sin(np.radians(i)), np.cos(np.radians(i))]
        ])

        R3_RAAN = np.array([
            [np.cos(np.radians(RAAN)), -np.sin(np.radians(RAAN)), 0],
            [np.sin(np.radians(RAAN)), np.cos(np.radians(RAAN)), 0],
            [0, 0, 1]
        ])

        # Combined rotation matrix
        R = R3_RAAN @ R1_i @ R3_omega

        # Apply rotation to each point
        rotated_plane = np.zeros((plane_x.shape[0], plane_x.shape[1], 3))
        for i in range(plane_x.shape[0]):
            for j in range(plane_x.shape[1]):
                point = np.array([plane_x[i, j], plane_y[i, j], plane_z[i, j]])
                rotated_point = R @ point
                rotated_plane[i, j, 0] = rotated_point[0]
                rotated_plane[i, j, 1] = rotated_point[1]
                rotated_plane[i, j, 2] = rotated_point[2]

        # Plot the orbital plane as a transparent surface
        ax.plot_surface(
            rotated_plane[:, :, 0],
            rotated_plane[:, :, 1],
            rotated_plane[:, :, 2],
            alpha=0.1, color='gray'
        )

    # Add equatorial plane
    theta = np.linspace(0, 2 * np.pi, 100)
    radius = np.linspace(0, max_val, 50)
    t, r = np.meshgrid(theta, radius)
    x = r * np.cos(t)
    y = r * np.sin(t)
    z = np.zeros_like(x)
    ax.plot_surface(x, y, z, alpha=0.1, color='cyan')

    # Set labels and title
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_zlabel('Z (km)')
    ax.set_title(f'GPS Satellite Orbit Visualization\n'
                 f'a = {a} km, e = {e}, i = {i}°, Ω = {RAAN}°, ω = {omega}°, M = {M}°')

    # Set equal aspect ratio
    ax.set_box_aspect([1, 1, 1])

    # Set limits
    max_val = max(max(abs(np.array(xs))), max(abs(np.array(ys))), max(abs(np.array(zs)))) * 1.2
    ax.set_xlim(-max_val, max_val)
    ax.set_ylim(-max_val, max_val)
    ax.set_zlim(-max_val, max_val)

    # Add legend
    ax.legend()

    if animate:
        # Animation function
        def update(frame):
            M_point = frame * 360 / 100
            r, _ = keplerian_to_cartesian(a, e, i, RAAN, omega, M_point)
            satellite.set_data([r[0]], [r[1]])
            satellite.set_3d_properties([r[2]])
            return satellite,

        # Create animation
        ani = FuncAnimation(fig, update, frames=100, interval=50, blit=True)
        plt.show()
        return ani
    else:
        plt.show()

    return fig

def add_full_gps_constellation(ax, a, e, i):
    """
    Add the full GPS constellation to the plot.

    Parameters:
    ax: Matplotlib 3D axis
    a: Semi-major axis (km)
    e: Eccentricity
    i: Inclination (degrees)
    """
    # GPS has 6 orbital planes with 4 satellites per plane (simplified)
    for plane_idx in range(6):
        RAAN = plane_idx * 60  # 6 planes equally spaced

        for sat_idx in range(4):
            M = sat_idx * 90  # 4 satellites equally spaced in each plane
            omega = 0  # For nearly circular orbits

            # Plot orbit for this plane
            xs, ys, zs = propagate_orbit(a, e, i, RAAN, omega, 0, num_points=200)
            ax.plot(xs, ys, zs, color='gray', alpha=0.3, linewidth=1)

            # Plot satellite
            r, _ = keplerian_to_cartesian(a, e, i, RAAN, omega, M)
            ax.plot([r[0]], [r[1]], [r[2]], 'ro', markersize=4, alpha=0.7)

def visualize_full_constellation():
    """
    Visualize the full GPS constellation.
    """
    # GPS parameters
    a = 26560  # km
    e = 0.01
    i = 55  # degrees

    # Create a figure
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Plot the Earth
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    x = R_earth * np.cos(u) * np.sin(v)
    y = R_earth * np.sin(u) * np.sin(v)
    z = R_earth * np.cos(v)
    ax.plot_surface(x, y, z, color='blue', alpha=0.3)

    # Add the GPS constellation
    add_full_gps_constellation(ax, a, e, i)

    # Add equatorial plane
    max_val = a * 1.2
    theta = np.linspace(0, 2 * np.pi, 100)
    radius = np.linspace(0, max_val, 50)
    t, r = np.meshgrid(theta, radius)
    x = r * np.cos(t)
    y = r * np.sin(t)
    z = np.zeros_like(x)
    ax.plot_surface(x, y, z, alpha=0.1, color='cyan')

    # Set labels and title
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_zlabel('Z (km)')
    ax.set_title('Full GPS Constellation Visualization')

    # Set equal aspect ratio
    ax.set_box_aspect([1, 1, 1])

    # Set limits
    ax.set_xlim(-max_val, max_val)
    ax.set_ylim(-max_val, max_val)
    ax.set_zlim(-max_val, max_val)

    plt.show()
    return fig

def interactive_console():
    """
    Run an interactive console to set orbital parameters and visualize the orbit.
    """
    print("Two-Body Orbital Mechanics Visualization - GPS")
    print("---------------------------------------------")

    # Default orbital parameters for GPS satellite
    default_a = 26560  # km (semi-major axis)
    default_e = 0.01   # eccentricity (nearly circular)
    default_i = 55     # degrees (inclination)
    default_RAAN = 0   # degrees (Right Ascension of the Ascending Node)
    default_omega = 0  # degrees (Argument of Perigee)
    default_M = 0      # degrees (Mean Anomaly)

    while True:
        print(f"\nCurrent GPS Orbital Parameters:")
        print(f"1. Semi-major axis (a): {default_a} km")
        print(f"2. Eccentricity (e): {default_e}")
        print(f"3. Inclination (i): {default_i}°")
        print(f"4. Right Ascension of Ascending Node (Ω): {default_RAAN}°")
        print(f"5. Argument of Perigee (ω): {default_omega}°")
        print(f"6. Mean Anomaly (M): {default_M}°")
        print("7. Visualize single GPS satellite orbit")
        print("8. Animate single GPS satellite orbit")
        print("9. Visualize full GPS constellation")
        print("10. Exit")

        choice = input("\nEnter your choice (1-10): ")

        if choice == '1':
            try:
                default_a = float(input("Enter new semi-major axis (km): "))
            except ValueError:
                print("Invalid input. Please enter a number.")

        elif choice == '2':
            try:
                default_e = float(input("Enter new eccentricity (0-1 for elliptical orbits): "))
            except ValueError:
                print("Invalid input. Please enter a number.")

        elif choice == '3':
            try:
                default_i = float(input("Enter new inclination (degrees): "))
            except ValueError:
                print("Invalid input. Please enter a number.")

        elif choice == '4':
            try:
                default_RAAN = float(input("Enter new RAAN (degrees): "))
            except ValueError:
                print("Invalid input. Please enter a number.")

        elif choice == '5':
            try:
                default_omega = float(input("Enter new argument of perigee (degrees): "))
            except ValueError:
                print("Invalid input. Please enter a number.")

        elif choice == '6':
            try:
                default_M = float(input("Enter new mean anomaly (degrees): "))
            except ValueError:
                print("Invalid input. Please enter a number.")

        elif choice == '7':
            try:
                visualize_orbit(default_a, default_e, default_i, default_RAAN, default_omega, default_M, animate=False)
                break
            except Exception as e:
                print(f"Error visualizing orbit: {e}")

        elif choice == '8':
            try:
                visualize_orbit(default_a, default_e, default_i, default_RAAN, default_omega, default_M, animate=True)
                break
            except Exception as e:
                print(f"Error animating orbit: {e}")

        elif choice == '9':
            try:
                visualize_full_constellation()
                break
            except Exception as e:
                print(f"Error visualizing constellation: {e}")

        elif choice == '10':
            print("Exiting program.")
            break

        else:
            print("Invalid choice. Please enter a number between 1 and 10.")

if __name__ == "__main__":
    interactive_console()

Two-Body Orbital Mechanics Visualization - GPS
---------------------------------------------

Current GPS Orbital Parameters:
1. Semi-major axis (a): 26560 km
2. Eccentricity (e): 0.01
3. Inclination (i): 55°
4. Right Ascension of Ascending Node (Ω): 0°
5. Argument of Perigee (ω): 0°
6. Mean Anomaly (M): 0°
7. Visualize single GPS satellite orbit
8. Animate single GPS satellite orbit
9. Visualize full GPS constellation
10. Exit


KeyboardInterrupt: Interrupted by user